In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Read de pickle
with open('../tracker_stubs/ball_detections.pkl', 'rb') as f:
    ballPositions = pickle.load(f)   

In [ ]:
ballPositions = [x.get(1,[]) for x in ballPositions]
# convert the list into pandas dataframe
dfBallPositions = pd.DataFrame(ballPositions,columns=['x1','y1','x2','y2'])

# interpolate the missing values
dfBallPositions = dfBallPositions.interpolate()
dfBallPositions = dfBallPositions.bfill()

In [ ]:
dfBallPositions

In [ ]:
dfBallPositions['mid_y'] = (dfBallPositions['y1'] + dfBallPositions['y2'])/2
dfBallPositions['mid_y_rolling_mean'] = dfBallPositions['mid_y'].rolling(window=5, min_periods=1, center=False).mean()


In [ ]:
#Plot
plt.plot(dfBallPositions['mid_y_rolling_mean'])

In [ ]:
dfBallPositions['delta_y'] = dfBallPositions['mid_y_rolling_mean'].diff()

In [ ]:
plt.plot(dfBallPositions['delta_y'])

In [ ]:
dfBallPositions['ball_hit'] = 0

In [ ]:
minimumChangeFramesForHit = 25
for i in range(1, len(dfBallPositions) - int(minimumChangeFramesForHit * 1.2)):
    negativePositionChange = dfBallPositions['delta_y'].iloc[i] > 0 and dfBallPositions['delta_y'].iloc[i + 1] < 0
    positivePositionChange = dfBallPositions['delta_y'].iloc[i] < 0 and dfBallPositions['delta_y'].iloc[i + 1] > 0

    if negativePositionChange or positivePositionChange:
        changeCount = 0
        for changeFrame in range(i + 1, i + int(minimumChangeFramesForHit * 1.2) + 1):
            negativePositionChangeFollowingFrame = dfBallPositions['delta_y'].iloc[i] > 0 and dfBallPositions['delta_y'].iloc[changeFrame] < 0
            positivePositionChangeFollowingFrame = dfBallPositions['delta_y'].iloc[i] < 0 and dfBallPositions['delta_y'].iloc[changeFrame] > 0

            if negativePositionChange and negativePositionChangeFollowingFrame:
                changeCount += 1
            elif positivePositionChange and positivePositionChangeFollowingFrame:
                changeCount += 1
    
        if changeCount > minimumChangeFramesForHit - 1:
            dfBallPositions['ball_hit'].iloc[i] = 1

frameNumsWithBallHits = dfBallPositions[dfBallPositions['ball_hit'] == 1].index.tolist()


In [ ]:
dfBallPositions[dfBallPositions['ball_hit'] == 1]
#frames where the ball is hit